### Synthetic EHR Data Generator

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- Configuration ---
NUM_RECORDS = 1500
START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2023, 12, 31)

# --- Synthetic Data Generation ---
def generate_synthetic_ehr_data():
    """
    Generates a synthetic EHR dataset as a pandas DataFrame.
    The dataset includes common issues like missing values, inconsistent formats,
    and incorrect data types to simulate real-world data cleaning challenges.
    """
    data = []
    patient_id_counter = 1001

    # Possible values for categorical data
    genders = ['Male', 'Female', 'Other', 'Unknown']
    # ICD-10 codes with intentional inconsistencies (whitespace, case, variations)
    diagnosis_codes = ['J45', 'J45.90', 'j45.909', 'I10', ' I10 ', 'E11.9', 'e11', 'N18.6', None, 'Z00.00']
    
    for _ in range(NUM_RECORDS):
        patient_id = patient_id_counter
        age = random.randint(18, 95)
        
        # Introduce some 'Unknown' or missing gender values
        gender = random.choices(genders, weights=[0.45, 0.45, 0.05, 0.05], k=1)[0]
        
        # Select a diagnosis code, allowing for None (missing)
        diagnosis_code = random.choice(diagnosis_codes)
        
        # Introduce missing lab results (NaN) and some outliers
        if random.random() < 0.15: # 15% chance of missing lab result
            lab_result = np.nan
        else:
            lab_result = round(random.uniform(50, 200) + random.choice([-30, 0, 40]), 2)

        # Generate random dates
        admission_date = START_DATE + timedelta(days=random.randint(0, (END_DATE - START_DATE).days))
        discharge_date = admission_date + timedelta(days=random.randint(1, 21))

        record = {
            'PatientID': patient_id,
            'Age': age if random.random() > 0.02 else str(age), # Introduce incorrect data type
            'Gender': gender,
            'DiagnosisCode': diagnosis_code,
            'LabResult': lab_result,
            'AdmissionDate': admission_date.strftime('%Y-%m-%d'),
            'DischargeDate': discharge_date.strftime('%Y-%m-%d %H:%M:%S') # Inconsistent date format
        }
        data.append(record)
        patient_id_counter += 1
        
    df = pd.DataFrame(data)
    
    # Shuffle the dataframe to make it look more realistic
    df = df.sample(frac=1).reset_index(drop=True)
    
    print("Synthetic EHR dataset generated successfully.")
    return df

if __name__ == '__main__':
    # Generate the data and save it to a CSV file
    synthetic_df = generate_synthetic_ehr_data()
    synthetic_df.to_csv('synthetic_ehr_data.csv', index=False)
    print(f"Dataset saved to 'synthetic_ehr_data.csv' with {len(synthetic_df)} records.")
    print("\nFirst 5 rows of the generated data:")
    print(synthetic_df.head())
    print("\nData Info:")
    synthetic_df.info()


Synthetic EHR dataset generated successfully.
Dataset saved to 'synthetic_ehr_data.csv' with 1500 records.

First 5 rows of the generated data:
   PatientID Age  Gender DiagnosisCode  LabResult AdmissionDate  \
0       1556  69    Male           e11     190.81    2023-05-08   
1       2053  72    Male         N18.6     186.75    2023-03-20   
2       1767  73  Female         N18.6        NaN    2022-04-27   
3       1284  54    Male         E11.9        NaN    2022-03-22   
4       1573  48  Female       j45.909      92.15    2022-10-14   

         DischargeDate  
0  2023-05-23 00:00:00  
1  2023-04-04 00:00:00  
2  2022-05-02 00:00:00  
3  2022-04-08 00:00:00  
4  2022-10-19 00:00:00  

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PatientID      1500 non-null   int64  
 1   Age            1500 non-null   object 
 2   Gend

### EHR Data Cleaning & Preprocessing Pipeline

In [2]:
import pandas as pd
import numpy as np

def clean_ehr_data(input_filepath='synthetic_ehr_data.csv'):
    """
    This function loads a raw EHR dataset, performs a series of cleaning and 
    preprocessing steps, and returns a cleaned DataFrame.

    Steps include:
    1.  Initial data inspection.
    2.  Handling missing values in 'DiagnosisCode' and 'LabResult'.
    3.  Standardizing 'DiagnosisCode' to a consistent format (e.g., ICD-10 main code).
    4.  Correcting data types for 'Age', 'AdmissionDate', and 'DischargeDate'.
    5.  Creating a new feature 'LengthOfStay' from date columns.
    6.  Standardizing 'Gender' categories.
    7.  Saving the cleaned data to a new file.
    """
    # --- 1. Load and Inspect Data ---
    print("Step 1: Loading and inspecting the raw data...")
    try:
        df = pd.read_csv(input_filepath)
    except FileNotFoundError:
        print(f"Error: The file '{input_filepath}' was not found. Please run generate_synthetic_data.py first.")
        return None

    print("\nInitial DataFrame Info:")
    df.info()
    print("\nInitial DataFrame Head:")
    print(df.head())
    print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")
    # Drop duplicates if any, though our generator doesn't create them
    df.drop_duplicates(inplace=True)

    # --- 2. Handle Missing Values ---
    print("\nStep 2: Handling missing values...")
    print("\nMissing values before cleaning:")
    print(df.isnull().sum())
    
    # Strategy for 'LabResult': Impute with the median
    # Median is often better than mean for skewed distributions or data with outliers.
    median_lab_result = df['LabResult'].median()
    df['LabResult'].fillna(median_lab_result, inplace=True)
    print(f"\nFilled missing 'LabResult' values with median: {median_lab_result:.2f}")

    # Strategy for 'DiagnosisCode': Fill with 'Unknown'
    # We can't guess a diagnosis, so marking it as unknown is a safe choice.
    df['DiagnosisCode'].fillna('Unknown', inplace=True)
    print("Filled missing 'DiagnosisCode' with 'Unknown'.")

    # --- 3. Standardize Medical Codes ---
    print("\nStep 3: Standardizing 'DiagnosisCode'...")
    # This process cleans and standardizes the codes:
    # 1. .str.strip(): Removes leading/trailing whitespace.
    # 2. .str.upper(): Converts to uppercase for consistency.
    # 3. .str.extract(r'([A-Z]\d{2})'): Uses a regular expression to extract the 
    #    core ICD-10 format (e.g., 'J45' from 'J45.909'). It looks for a letter
    #    followed by two digits.
    df['StandardizedDiagnosisCode'] = df['DiagnosisCode'].str.strip().str.upper().str.extract(r'([A-Z]\d{2})')
    # If the regex fails (e.g., for 'Unknown'), fill the result with 'Unknown'
    df['StandardizedDiagnosisCode'].fillna('UNKNOWN', inplace=True)
    
    print("Created 'StandardizedDiagnosisCode' column with cleaned codes.")
    print("Value counts of standardized codes:")
    print(df['StandardizedDiagnosisCode'].value_counts().head())

    # --- 4. Correct Data Types ---
    print("\nStep 4: Correcting data types...")
    # Convert 'Age' to a numeric type, coercing errors to NaN and then filling
    df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
    df['Age'].fillna(df['Age'].median(), inplace=True) # Fill any conversion errors with median age
    df['Age'] = df['Age'].astype(int)
    print("Corrected 'Age' column to integer type.")

    # Convert date columns to datetime objects
    # The `errors='coerce'` will turn any unparseable dates into NaT (Not a Time)
    df['AdmissionDate'] = pd.to_datetime(df['AdmissionDate'], errors='coerce')
    df['DischargeDate'] = pd.to_datetime(df['DischargeDate'], errors='coerce')
    print("Corrected date columns to datetime type.")
    
    # Handle any potential NaT values from conversion errors, though unlikely with our data
    if df['AdmissionDate'].isnull().any() or df['DischargeDate'].isnull().any():
        print("Warning: Some dates could not be parsed and were set to NaT.")
        df.dropna(subset=['AdmissionDate', 'DischargeDate'], inplace=True) # Drop rows with invalid dates

    # --- 5. Feature Engineering: Length of Stay ---
    print("\nStep 5: Engineering 'LengthOfStay' feature...")
    # Calculate the difference between discharge and admission dates
    df['LengthOfStay'] = (df['DischargeDate'] - df['AdmissionDate']).dt.days
    # Ensure length of stay is non-negative
    df = df[df['LengthOfStay'] >= 0]
    print("Created 'LengthOfStay' column (in days).")

    # --- 6. Standardize Categorical Data ---
    print("\nStep 6: Standardizing 'Gender' column...")
    gender_map = {
        'Male': 'M',
        'Female': 'F',
        'Other': 'O',
        'Unknown': 'U'
    }
    df['Gender'] = df['Gender'].map(gender_map).fillna('U')
    print("Standardized 'Gender' to single-letter codes.")
    print(df['Gender'].value_counts())
    
    # --- 7. Final Review and Save ---
    print("\nStep 7: Finalizing and saving the cleaned data...")
    # Select and reorder columns for the final dataset
    final_columns = [
        'PatientID', 
        'Age', 
        'Gender',
        'StandardizedDiagnosisCode', 
        'LabResult', 
        'AdmissionDate', 
        'DischargeDate', 
        'LengthOfStay'
    ]
    cleaned_df = df[final_columns]
    
    print("\nCleaned DataFrame Info:")
    cleaned_df.info()
    print("\nCleaned DataFrame Head:")
    print(cleaned_df.head())
    
    # Save the cleaned dataframe
    output_filepath = 'ehr_data_cleaned.csv'
    cleaned_df.to_csv(output_filepath, index=False)
    print(f"\nSuccessfully cleaned the data. Output saved to '{output_filepath}'.")
    
    return cleaned_df

if __name__ == '__main__':
    clean_ehr_data()


Step 1: Loading and inspecting the raw data...

Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PatientID      1500 non-null   int64  
 1   Age            1500 non-null   int64  
 2   Gender         1500 non-null   object 
 3   DiagnosisCode  1352 non-null   object 
 4   LabResult      1269 non-null   float64
 5   AdmissionDate  1500 non-null   object 
 6   DischargeDate  1500 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 82.2+ KB

Initial DataFrame Head:
   PatientID  Age  Gender DiagnosisCode  LabResult AdmissionDate  \
0       1556   69    Male           e11     190.81    2023-05-08   
1       2053   72    Male         N18.6     186.75    2023-03-20   
2       1767   73  Female         N18.6        NaN    2022-04-27   
3       1284   54    Male         E11.9        NaN    2022-03-22  

C:\Users\HP\AppData\Local\Temp\ipykernel_14332\528108779.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LabResult'].fillna(median_lab_result, inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_14332\528108779.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex